In [5]:
#NEED TO BE RUN TWICE (Unknown error)
!pip install xarray
!pip install cartopy
!pip install pyreadr

In [6]:
import numpy as np
import pandas as pd
import pickle
import os
import random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import scipy.stats as stats
import xarray as xr
import zipfile
from dateutil.relativedelta import relativedelta
import scipy.stats as stats
from scipy.optimize import minimize
import plotly.graph_objs as go
import pyreadr
import math
import time
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import s3fs
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "zhippofficiel"

In [8]:
# Statistical models and function for modelization of precipitations.
# Include GEV and EGPD functions.

def H(xi,X):
    X=np.array(X)
    return((1+xi*X)**(-1/xi))

def F(params,X):
    kappa,sigma,xi = params
    X=np.array(X)
    return(H(xi,X/sigma)**kappa)

def f(params,X):
    kappa,sigma,xi = params
    return((kappa/sigma)*((1+((xi/sigma)*X))**(-1-(kappa/xi))))

def minus_l_f(params,X):
    kappa,sigma,xi = params
    return -np.sum(list(np.log((kappa/sigma)*((1+((xi/sigma)*X))**(-1-(kappa/xi))))))

def H_inv(xi,X):
    X=np.array(X)
    return(((X**(-xi))-1)/xi)

def Q(params,p):
    kappa,sigma,xi = params
    p = np.array(p)
    return((sigma/xi)*((p**(-xi/kappa))-1))

def data_xhour(data,x):
    results = []
    N=len(data)
    for i in range(N//x):
        value = 0
        for j in range(x):
            value = value + data[i*x+j]
        results.append(value)
    return results

def get_optimal_params(lat,lon,duration,treshold): #lat,lon in '41.25' format; duration in hour (only int) ; treshold in m
    selected_value = list(full_dataset.sel(lon=lon, lat=lat).values)
    list_precip = np.array(selected_value)
    data_dur = np.array(data_xhour(list_precip,duration))
    cleaned_data = np.array([(x-treshold)*1000 for x in data_dur if x>treshold])
    initial_guess = [0.5, 1, 0.3]
    objective_function = lambda params: minus_l_f(params, cleaned_data)
    result_minimize = minimize(objective_function, initial_guess, method='Nelder-Mead')
    optimal_params = result_minimize.x
    return optimal_params

def file_oneD_all_points(duration,treshold):
    Store_params = []
    lons = list(full_dataset['lon'].values)
    lats = list(full_dataset['lat'].values)
    for lat in lats:
        print('Starting to work on points with latitude {}°N'.format(float(lat)))
        for lon in lons:
            optimal_params  = get_optimal_params(lat,lon,duration,treshold)
            res = list(optimal_params)
            res.append(lat)
            res.append(lon)
            Store_params.append(res)
    return Store_params

def create_save_optimal_params(durations,treshold): #durations in hours ([int] only) ; treshold in m 
    FILEDIR_KEY_OUT_S3 = "ExtremeValuesPrecipitations/ERA Analysis/Optimal_parameters"       
    print(" ------------------------- Starting searching optimal parameters. ------------------------- ")
    for duration in durations:
        start_time = time.time()
        print(" ------------------------- For a period of {} hours. ------------------------- ".format(duration))
        FILE_KEY_OUT_S3 = FILEDIR_KEY_OUT_S3+"/optimal_params_"+str(duration)+"_hours.txt"
        optimal_params_oneD = file_oneD_all_points(duration,treshold)
        optimal_params_oneD_list=[]
        for x in optimal_params_oneD:
            value = list(x)
            optimal_params_oneD_list.append(value)
            column_names = ['kappa', 'sigma', 'xi','lat','lon']
        Optimal_params_df = pd.DataFrame(optimal_params_oneD_list, columns=column_names)
        FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3
        with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
            Optimal_params_df.to_csv(file_out, sep='\t', index=False)
        print(' --- Saving File Completed. ---')
        end_time = time.time()
        elapsed_time = end_time - start_time
        print("Time taken to compute data: {:.2f} seconds".format(elapsed_time))
    return("Task completed successfully!")

def return_levels_to_quantiles(return_levels,duration): # return_level en années ; duration en heures 
    return_levels = np.array(return_levels)
    res = (np.array([(duration/(365*time*24)) for time in return_levels]))
    return res

def IDF_one_D(params,return_levels,duration):
    quantiles = return_levels_to_quantiles(return_levels,duration)
    Qvalues = Q(params,quantiles)
    return Qvalues

def get_return_levels_int_or_cumul(lat,lon,durations,return_levels,Int):
    FILEDIR_KEY_S3 = "ExtremeValuesPrecipitations/ERA Analysis/Optimal_parameters"      
    Return_Values = [] 
    for dur in durations:
        FILE_KEY_S3 = FILEDIR_KEY_S3 + '/optimal_params_'+str(dur)+'_hours.txt'
        FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3
        with fs.open(FILE_PATH_S3, mode="rb") as file_in:
            df = pd.read_csv(file_in, sep='\t')
        selected_point = df[(df['lat'] == lat) & (df['lon'] == lon)]
        optimal_params = [float(selected_point['kappa']),float(selected_point['sigma']),float(selected_point['xi'])]
        Qvalues = IDF_one_D(optimal_params,return_levels,dur)
        Return_Values.append(Qvalues)
    plt.figure()
    for i in range(len(return_levels)):
        values = []
        for j in range(len(durations)):
            if Int:
                values.append(Return_Values[j][i]/durations[j])
            if not Int:
                values.append(Return_Values[j][i])
        plt.plot(durations,values,marker='s',label='Return levels over {} Year'.format(return_levels[i]))
    plt.xticks(durations)
    plt.xlim(0.6,15)
    #plt.xscale('log')
    #plt.yscale('log')
    plt.grid()
    #plt.legend()
    plt.xlabel('Durations (hours)')
    if Int:
        plt.ylabel('Intensity (mm/hour)')
    if not Int:
        plt.ylabel('Cumuls (mm)')
    plt.title('IDF Curve')
    plt.show()
    return Return_Values

In [27]:
# Opening full dataset

lons = ['0', '0.25', '0.5', '0.75', '1', '1.25', '1.5', '1.75', '2',
       '2.25', '2.5', '2.75', '3', '3.25', '3.5', '3.75', '4', '4.25',
       '4.5', '4.75', '5', '5.25', '5.5', '5.75', '6', '6.25', '6.5',
       '6.75', '7', '7.25', '7.5', '7.75', '8', '8.25', '8.5', '8.75',
       '9', '9.25', '9.5', '9.75', '10', '-6', '-5.75', '-5.5', '-5.25',
       '-5', '-4.75', '-4.5', '-4.25', '-4', '-3.75', '-3.5', '-3.25',
       '-3', '-2.75', '-2.5', '-2.25', '-2', '-1.75', '-1.5', '-1.25',
       '-1', '-0.75', '-0.5', '-0.25']

lats = ['41', '41.25', '41.5', '41.75', '42', '42.25', '42.5', '42.75',
       '43', '43.25', '43.5', '43.75', '44', '44.25', '44.5', '44.75',
       '45', '45.25', '45.5', '45.75', '46', '46.25', '46.5', '46.75',
       '47', '47.25', '47.5', '47.75', '48', '48.25', '48.5', '48.75',
       '49', '49.25', '49.5', '49.75', '50', '50.25', '50.5', '50.75',
       '51', '51.25', '51.5', '51.75', '52']

ids = np.linspace(0,len(lons)-1,len(lons))

In [30]:
FILEDIR_KEY_S3 = "ExtremeValuesPrecipitations/ERA DATA/CSV_files"
FILE_KEY_S3 = FILEDIR_KEY_S3 + '/LON_0.csv'
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3
with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_csv(file_in)

In [31]:
df

,0,1,2,3,4,5,6,7,8,9,...,382766,382767,382768,382769,382770,382771,382772,382773,382774,382775
0,9.713266e-06,0.000087,7.255810e-04,0.000249,7.867745e-05,2.234051e-05,9.713266e-07,0.000015,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.456990e-05,0.000060,2.661435e-04,0.000480,2.748854e-04,3.108245e-05,9.713266e-07,0.000011,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,4.176705e-05,0.000089,6.333049e-04,0.000488,6.235917e-04,4.565235e-05,7.770613e-06,0.000010,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,7.090684e-05,0.000117,1.418137e-04,0.000203,1.777528e-04,4.468102e-05,4.273837e-05,0.000007,9.713266e-07,9.713266e-07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.379284e-04,0.000155,1.602689e-04,0.000220,1.699822e-04,1.563836e-04,1.534696e-04,0.000020,1.165592e-05,9.713266e-06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2.700288e-04,0.000285,3.428783e-04,0.000373,3.370503e-04,4.380683e-04,3.846453e-04,0.000093,6.022225e-05,6.119358e-05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,4.914913e-04,0.000594,7.148964e-04,0.000746,8.265990e-04,1.029606e-03,9.888104e-04,0.000273,1.524983e-04,1.447277e-04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,7.440362e-04,0.000793,9.606420e-04,0.001123,1.404538e-03,1.750331e-03,1.784327e-03,0.000454,4.613801e-04,8.780793e-04,...,0.000000,0.000026,0.000028,0.000005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,6.507888e-04,0.000762,9.606420e-04,0.001180,1.565779e-03,2.306901e-03,1.965965e-03,0.000417,4.574948e-04,7.586061e-04,...,0.000000,0.000023,0.000023,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,4.302977e-04,0.000603,9.722979e-04,0.001168,1.886316e-03,2.436087e-03,2.355467e-03,0.000539,2.913980e-04,1.185018e-03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}, index=['x', 'y', 'z'])
xarray_df = xr.DataArray.from_series(df.stack())
print(xarray_df)

<xarray.DataArray (level_0: 3, level_1: 2)> Size: 48B
array([[1, 4],
       [2, 5],
       [3, 6]])
Coordinates:
  * level_0  (level_0) object 24B 'x' 'y' 'z'
  * level_1  (level_1) object 16B 'A' 'B'


In [33]:
xarray_df

<xarray.DataArray (level_0: 3, level_1: 2)> Size: 48B
array([[1, 4],
       [2, 5],
       [3, 6]])
Coordinates:
  * level_0  (level_0) object 24B 'x' 'y' 'z'
  * level_1  (level_1) object 16B 'A' 'B'